# Are professional forecastors rational?

## Introduction

###### Forecasting is simply a prediction of an economic metric, based on all available information. This is an integral part of decision-making for all economic agents, from firms, to government entities, to individuals. A simple example to illustrate the impact a forecast would have is the price of a specific stock, if there is a forecast that indicates a decrease in price, selling or shorting the stock would be the optimal move. Another example would be an interest rate forecast, if there appears to be a significant increase, firms would be hesitant to borrow large sums of money for new projects. Economists and other professionals are constantly improving methods of forecasting every year, in an attempt to perfectly predict the future state of the world. But people have been forecasting large economic metrics, such as GDP and inflation, for a long period of time. By gathering old forecasts of these metrics, and using hindsight to directly observe the actual values, it should be interesting to see how accurate these predictions were. More importantly, it would be interesting to see if these forecasts were rational. This analysis will not be looking for identical statistics. Rather, it will observe if these forecasts adhere to the true probability distributions of the economy, and how these individual forecasters react to new information.

## Analysis

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
ngdp=pd.read_excel(r'\Users\barry\Desktop\NGDP.xlsx')
pgdp=pd.read_excel(r'\Users\barry\Desktop\PGDP.xlsx')
# windows

C:\Users\barry\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\barry\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
# ngdp=pd.read_excel('/Users/barryzou/Desktop/NGDP.xlsx')
# pgdp=pd.read_excel('/Users/barryzou/Desktop/PGDP.xlsx')
# mac

### Nominal GDP Growth Rate

In [4]:
a=ngdp.groupby(['YEAR', 'QUARTER'])['NGDP1'].apply(lambda x: x.mode().iloc[0]).reset_index() 
#create dataframe with historical values of NGDP
#use mode since there were the occassional value that was different than the others, assume typo

In [5]:
f_row = {'YEAR': 1968, 'QUARTER': 3, 'NGDP1': None}
a2 = pd.concat([pd.DataFrame([f_row]), a], ignore_index=True)
a2['NGDP1'] = a2['NGDP1'].shift(-1)
a2 = a2[:-1]
#shift all historical NGDP values up to match time period, add empty row to top, remove bottom row

In [6]:
a2['Date'] = pd.PeriodIndex(year=a2['YEAR'], quarter=a2['QUARTER']).to_timestamp()
a2.drop(columns=['YEAR','QUARTER'],inplace=True)
a2.set_index(['Date'],inplace=True)
a2.rename(columns={'NGDP1': 'Nominal GDP'}, inplace=True)
#replace year and quarter column with timestamp index, rename 'NGDP1' column

In [7]:
a2['GDP Growth'] = a2['Nominal GDP'].pct_change() * 100
#adding column that calculates percent change

In [8]:
a2.drop('Nominal GDP', axis=1, inplace=True)
#remove the nominal GDP column now, it's no longer needed, guess there was no point renaming it

In [9]:
ngdp['Date'] = pd.PeriodIndex(year=ngdp['YEAR'], quarter=ngdp['QUARTER']).to_timestamp()
ngdp.drop(columns=['YEAR','QUARTER'],inplace=True)
ngdp.set_index(['Date'],inplace=True)
#set the entire dataframe index to timestamp

In [10]:
fcid_ngdp={}
for i in range(1, 605):
    fcid_ngdp[f'fc{i}']=ngdp[ngdp['ID'] == i]
    fcid_ngdp[f'fc{i}']=fcid_ngdp[f'fc{i}'].drop(['INDUSTRY', 'NGDPA', 'NGDPB'], axis=1)
    fcid_ngdp[f'fc{i}'].rename(columns={'NGDP1': 'Previous Q NGDP', 'NGDP2': 'Forecast 0', 'NGDP3': 'Forecast 1', 'NGDP4': 'Forecast 2', 'NGDP5': 'Forecast 3', 'NGDP6': 'Forecast 4'}, inplace=True)
#build a dictionary that can access dataframes of specific forecastors by ID, drop irrelevant columns, rename kept columns

In [11]:
for i in range(1, 605):
    fcid_ngdp[f'fc{i}']['Growth Forecast'] = ((fcid_ngdp[f'fc{i}']['Forecast 0'] - fcid_ngdp[f'fc{i}']['Previous Q NGDP']) / fcid_ngdp[f'fc{i}']['Previous Q NGDP']) * 100
    fcid_ngdp[f'fc{i}']['GF +1'] = ((fcid_ngdp[f'fc{i}']['Forecast 1'] - fcid_ngdp[f'fc{i}']['Forecast 0']) / fcid_ngdp[f'fc{i}']['Forecast 0']) * 100
    fcid_ngdp[f'fc{i}']['GF +2'] = ((fcid_ngdp[f'fc{i}']['Forecast 2'] - fcid_ngdp[f'fc{i}']['Forecast 1']) / fcid_ngdp[f'fc{i}']['Forecast 1']) * 100
    fcid_ngdp[f'fc{i}']['GF +3'] = ((fcid_ngdp[f'fc{i}']['Forecast 3'] - fcid_ngdp[f'fc{i}']['Forecast 2']) / fcid_ngdp[f'fc{i}']['Forecast 2']) * 100
    fcid_ngdp[f'fc{i}']['GF +4'] = ((fcid_ngdp[f'fc{i}']['Forecast 4'] - fcid_ngdp[f'fc{i}']['Forecast 3']) / fcid_ngdp[f'fc{i}']['Forecast 3']) * 100
    fcid_ngdp[f'fc{i}']=fcid_ngdp[f'fc{i}'].drop(['Previous Q NGDP', 'Forecast 0', 'Forecast 1', 'Forecast 2', 'Forecast 3', 'Forecast 4'], axis=1)
#create new columns, "Growth Forecast", "GF +1",...,"GF +4", which are forecasts of economic growth
#drop all columns with just GDP forecasts, not needed

In [12]:
f_col=['Growth Forecast', 'GF +1', 'GF +2', 'GF +3', 'GF +4']
fe_col=['FE1', 'FE2', 'FE3', 'FE4', 'FE5']
#columns we will be working with, then dropping

In [13]:
def shift_forecasts(id):
    fcid_ngdp2[f'fc{id}']['GF +1']=fcid_ngdp2[f'fc{id}']['GF +1'].shift(1)
    fcid_ngdp2[f'fc{id}']['GF +2']=fcid_ngdp2[f'fc{id}']['GF +2'].shift(2)
    fcid_ngdp2[f'fc{id}']['GF +3']=fcid_ngdp2[f'fc{id}']['GF +3'].shift(3)
    fcid_ngdp2[f'fc{id}']['GF +4']=fcid_ngdp2[f'fc{id}']['GF +4'].shift(4)
#a function that will shift growth rate forecasts to the time period it is forecasting

In [14]:
def calc_FE(id):
    for i in range(1,6):
        fcid_ngdp2[f'fc{id}'][f'FE{i}']=fcid_ngdp2[f'fc{id}']['GDP Growth']-fcid_ngdp2[f'fc{id}']['Growth Forecast']
        fcid_ngdp2[f'fc{id}'][f'FE{i}']=fcid_ngdp2[f'fc{id}']['GDP Growth']-fcid_ngdp2[f'fc{id}']['GF +1']
        fcid_ngdp2[f'fc{id}'][f'FE{i}']=fcid_ngdp2[f'fc{id}']['GDP Growth']-fcid_ngdp2[f'fc{id}']['GF +2']
        fcid_ngdp2[f'fc{id}'][f'FE{i}']=fcid_ngdp2[f'fc{id}']['GDP Growth']-fcid_ngdp2[f'fc{id}']['GF +3']
        fcid_ngdp2[f'fc{id}'][f'FE{i}']=fcid_ngdp2[f'fc{id}']['GDP Growth']-fcid_ngdp2[f'fc{id}']['GF +4']
#this function will create columns that have forecast errors, most will be four forecast errors per period

In [15]:
fcid_ngdp2={}
for i in range(1, 605):
    fcid_ngdp2[f'fc{i}']=pd.merge(a2, fcid_ngdp[f'fc{i}'], on='Date', how='left')
    shift_forecasts(i)
    fcid_ngdp2[f'fc{i}']['News'] = fcid_ngdp2[f'fc{i}'][f_col].apply(lambda row: row.max() - row.min(), axis=1)
    calc_FE(i)
    fcid_ngdp2[f'fc{i}']['Avg_FE']=fcid_ngdp2[f'fc{i}'][fe_col].mean(axis=1, skipna=True)
    fcid_ngdp2[f'fc{i}'].drop(columns=f_col, inplace=True)
    fcid_ngdp2[f'fc{i}'].drop(columns=fe_col, inplace=True)
#here is a second dictionary for each forecaster, with actual GDP growth, a news column, and a column for average forecast error
#columns were added and removed after use

In [16]:
m_col=['News', 'Avg_FE']
n_reg={
    'ID':[],
    'beta1':[],
    'SE':[],
    'P-Value':[]
}
p_reg={
    'ID':[],
    'beta1':[],
    'SE':[],
    'P-Value':[]
}
#columns to clean, a dictionary to record regression information

In [17]:
fcid_ngdp3={}
for i in range(1, 605):
    fcid_ngdp3[f'fc{i}']=fcid_ngdp2[f'fc{i}'].dropna(subset=m_col)
    if len(fcid_ngdp3[f'fc{i}'])<2:
        continue
    X = sm.add_constant(fcid_ngdp3[f'fc{i}']['News'])
    model = sm.OLS(fcid_ngdp3[f'fc{i}']['Avg_FE'], X).fit()
    beta1 = model.params
    se = model.bse
    p_val = model.pvalues
    n_reg['ID'].append(i)
    n_reg['beta1'].append(beta1[1])
    n_reg['SE'].append(se[1])
    n_reg['P-Value'].append(p_val[1])

C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaco

In [18]:
n_beta=pd.DataFrame(n_reg)

In [19]:
rat_ngdp = (n_beta['P-Value']>0.5).sum()
print(f'{rat_ngdp} out of 300 forecasters forecasting nominal GDP growth are rational.')

89 out of 300 forecasters forecasting nominal GDP growth are rational.


### Inflation

In [20]:
b=pgdp.groupby(['YEAR', 'QUARTER'])['PGDP1'].apply(lambda x: x.mode().iloc[0]).reset_index() 
#steps for this variable will be fairly identical to GDP growth

In [21]:
f2_row = {'YEAR': 1968, 'QUARTER': 3, 'PGDP1': None}
b2 = pd.concat([pd.DataFrame([f2_row]), b], ignore_index=True)
b2['PGDP1'] = b2['PGDP1'].shift(-1)
b2 = b2[:-1]

In [22]:
b2['Date'] = pd.PeriodIndex(year=b2['YEAR'], quarter=b2['QUARTER']).to_timestamp()
b2.drop(columns=['YEAR','QUARTER'],inplace=True)
b2.set_index(['Date'],inplace=True)

In [23]:
b2['Inflation'] = b2['PGDP1'].pct_change() * 100

In [24]:
b2.drop('PGDP1', axis=1, inplace=True)

In [25]:
pgdp['Date'] = pd.PeriodIndex(year=pgdp['YEAR'], quarter=pgdp['QUARTER']).to_timestamp()
pgdp.drop(columns=['YEAR','QUARTER'],inplace=True)
pgdp.set_index(['Date'],inplace=True)

In [26]:
fcid_pgdp={}
for i in range(1, 605):
    fcid_pgdp[f'fc{i}']=pgdp[pgdp['ID'] == i]
    fcid_pgdp[f'fc{i}']=fcid_pgdp[f'fc{i}'].drop(['INDUSTRY', 'PGDPA', 'PGDPB'], axis=1)
    fcid_pgdp[f'fc{i}'].rename(columns={'PGDP1': 'Previous Q PGDP', 'PGDP2': 'Forecast 0', 'PGDP3': 'Forecast 1', 'PGDP4': 'Forecast 2', 'PGDP5': 'Forecast 3', 'PGDP6': 'Forecast 4'}, inplace=True)

In [27]:
for i in range(1, 605):
    fcid_pgdp[f'fc{i}']['Growth Forecast'] = ((fcid_pgdp[f'fc{i}']['Forecast 0'] - fcid_pgdp[f'fc{i}']['Previous Q PGDP']) / fcid_pgdp[f'fc{i}']['Previous Q PGDP']) * 100
    fcid_pgdp[f'fc{i}']['GF +1'] = ((fcid_pgdp[f'fc{i}']['Forecast 1'] - fcid_pgdp[f'fc{i}']['Forecast 0']) / fcid_pgdp[f'fc{i}']['Forecast 0']) * 100
    fcid_pgdp[f'fc{i}']['GF +2'] = ((fcid_pgdp[f'fc{i}']['Forecast 2'] - fcid_pgdp[f'fc{i}']['Forecast 1']) / fcid_pgdp[f'fc{i}']['Forecast 1']) * 100
    fcid_pgdp[f'fc{i}']['GF +3'] = ((fcid_pgdp[f'fc{i}']['Forecast 3'] - fcid_pgdp[f'fc{i}']['Forecast 2']) / fcid_pgdp[f'fc{i}']['Forecast 2']) * 100
    fcid_pgdp[f'fc{i}']['GF +4'] = ((fcid_pgdp[f'fc{i}']['Forecast 4'] - fcid_pgdp[f'fc{i}']['Forecast 3']) / fcid_pgdp[f'fc{i}']['Forecast 3']) * 100
    fcid_pgdp[f'fc{i}']=fcid_pgdp[f'fc{i}'].drop(['Previous Q PGDP', 'Forecast 0', 'Forecast 1', 'Forecast 2', 'Forecast 3', 'Forecast 4'], axis=1)

In [28]:
def shift_forecasts_p(id):
    fcid_pgdp2[f'fc{id}']['GF +1']=fcid_pgdp2[f'fc{id}']['GF +1'].shift(1)
    fcid_pgdp2[f'fc{id}']['GF +2']=fcid_pgdp2[f'fc{id}']['GF +2'].shift(2)
    fcid_pgdp2[f'fc{id}']['GF +3']=fcid_pgdp2[f'fc{id}']['GF +3'].shift(3)
    fcid_pgdp2[f'fc{id}']['GF +4']=fcid_pgdp2[f'fc{id}']['GF +4'].shift(4)
#probably more effecient to make this function take dataframe as an input, but it was less effort to write a new one for PGDP

In [29]:
def calc_FE_p(id):
    for i in range(1,6):
        fcid_pgdp2[f'fc{id}'][f'FE{i}']=fcid_pgdp2[f'fc{id}']['Inflation']-fcid_pgdp2[f'fc{id}']['Growth Forecast']
        fcid_pgdp2[f'fc{id}'][f'FE{i}']=fcid_pgdp2[f'fc{id}']['Inflation']-fcid_pgdp2[f'fc{id}']['GF +1']
        fcid_pgdp2[f'fc{id}'][f'FE{i}']=fcid_pgdp2[f'fc{id}']['Inflation']-fcid_pgdp2[f'fc{id}']['GF +2']
        fcid_pgdp2[f'fc{id}'][f'FE{i}']=fcid_pgdp2[f'fc{id}']['Inflation']-fcid_pgdp2[f'fc{id}']['GF +3']
        fcid_pgdp2[f'fc{id}'][f'FE{i}']=fcid_pgdp2[f'fc{id}']['Inflation']-fcid_pgdp2[f'fc{id}']['GF +4']
#same rationale for this function

In [31]:
fcid_pgdp2={}
for i in range(1, 605):
    fcid_pgdp2[f'fc{i}']=pd.merge(b2, fcid_pgdp[f'fc{i}'], on='Date', how='left')
    shift_forecasts_p(i)
    fcid_pgdp2[f'fc{i}']['News'] = fcid_pgdp2[f'fc{i}'][f_col].apply(lambda row: row.max() - row.min(), axis=1)
    calc_FE_p(i)
    fcid_pgdp2[f'fc{i}']['Avg_FE']=fcid_pgdp2[f'fc{i}'][fe_col].mean(axis=1, skipna=True)
    fcid_pgdp2[f'fc{i}'].drop(columns=f_col, inplace=True)
    fcid_pgdp2[f'fc{i}'].drop(columns=fe_col, inplace=True)

In [32]:
fcid_pgdp3={}
for i in range(1, 605):
    fcid_pgdp3[f'fc{i}']=fcid_pgdp2[f'fc{i}'].dropna(subset=m_col)
    if len(fcid_pgdp3[f'fc{i}'])<2:
        continue
    X = sm.add_constant(fcid_pgdp3[f'fc{i}']['News'])
    model = sm.OLS(fcid_pgdp3[f'fc{i}']['Avg_FE'], X).fit()
    beta1 = model.params
    se = model.bse
    p_val = model.pvalues
    p_reg['ID'].append(i)
    p_reg['beta1'].append(beta1[1])
    p_reg['SE'].append(se[1])
    p_reg['P-Value'].append(p_val[1])

C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid
C:\Users\barry\anacon

In [33]:
p_beta=pd.DataFrame(p_reg)

In [34]:
rat_pgdp = (p_beta['P-Value']>0.5).sum()
print(f'{rat_pgdp} out of 300 forecasters forecasting inflation are rational')

103 out of 300 forecasters forecasting inflation are rational


## Discussion

###### While attempting to answer this question, there were certainly limitations that range from missing data to python language fluency. The ultimate goal was to compare forecasts of each individual forecaster to the actual value, which was done with nominal GDP growth and inflation. The steps that were taken for both are almost identical, so discussion will only pertain to nominal GDP growth to avoid redundancy. The first limitation was finding data on FRED that matches the methodology of data in the spreadsheet. After some attempts of searching, and even one attempt to calculate it myself, it dawned on me that it might be simpler to just use the historical values given in the first column of every worksheet. The documentation explains that it represents the historical value of the previous quarter, so a new dataframe was created and the first column was shifted up so that the value matched the year. Then using the same dataframe, a new column was added to calculate growth rate.


###### Now, to evaluate the rationality of each individual forecaster, dataframes were created for each one, ranging from ID 1-604. Dictionaries were especially helpful for this since a loop could handle the creation of all the dataframes. This method would be employed two more times as more changes were made to the forecaster’s individual dataframes. Since forecasters would generally make five forecasts, for the current period up to one year ahead, it seemed like the correct move to calculate the quarterly change of each of them, which essentially would be a forecast of the GDP growth during that interval of time. This means that if a forecaster records their forecast for each quarter, which they didn’t, they would have four forecasts of growth for a single quarter, all made at different points in time. Columns to collect forecast errors were made, a maximum of four, and an average for forecast error was created in a separate column.


###### News was a difficult variable to calculate. The first idea was to calculate the absolute value of all differences in growth rate forecasts, and use the sum. But several forecasters were very inconsistent with their forecasts, and often did not report their forecasts every single quarter. But did that necessarily make them less reactive to news? The forecasters who were diligent would end up with a greater news value, just by virtue of making more forecasts. In the end, it was settled on range, the difference between the largest forecast and the smallest. This would simply demonstrate the magnitude of change that will occur, which should demonstrate reaction levels since all forecasters are assumed to be privy to the same knowledge.


###### Missing data was an issue when running the regression, and there were cases when forecasters also only had one row of data. Both of these needed to be accounted for when building a model for average forecast error with one predictor, news. A loop with a dictionary was employed to create an OLS model summary for each forecaster, and important information, such as the coefficient, standard error, and p-value was added to a new dataframe. To test forecaster rationality, the null hypothesis, the coefficient of news is zero, needed to be accepted. Using the dataframe with the regression info, there was a sum of forecasters with a p-value greater than 0.05, as the test will be at a 5% level of significance. The result is printed in the analysis section, and will be elaborated upon in the conclusion. 


## Conclusion

###### 89 out of 300 forecasters forecasting nominal GDP growth are rational, 103 out of 300 forecasters forecasting inflation are rational, these are the statements printed at the bottom of my analysis section, 29.6% and 34.3% respectively. Without context as to how the broader industry of economic forecaster looks like, this does not seem like a very high percentage. It is clear that the majority of forecasters are overreacting to information from media sources or other individuals. The results did contradict my initial thoughts as to what the data would tell. Historically, inflation has been a metric that evoked more emotional responses than GDP growth, since many aspects of people’s lives are more directly affected by inflation than GDP. Think gas prices, groceries, and other daily improvements. But the percentage difference, by eye test, does not seem too great. Is it significant? That’s a project for another time.
